In [32]:
import sys, os
from pathlib import Path

# Adicione o diretório do projeto ao sys.path
sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

In [33]:
from kinematicsrobotics.datahandler import Save, Extract
from kinematicsrobotics.kinematics import Robo, Spacemapping
from numpy import array, append
from pandas import DataFrame

# Cinemática Direta

In [34]:
Elos = [['theta_1',10,0,90,0],
            ['theta_2',0,18,180,0],
            ['theta_3',0,18,-180,0],
            ['theta_4',0,0,90,90],
            ['theta_5',18,0,0,0]]

# Cria o robo por meio dos parâmetros
robo = Robo('Manipulador Didático',Elos)

mapping = Spacemapping(robo=robo)


# Cinemática Inversa

## Base de dados

In [35]:
def partition(dataset,axis):
    return array(dataset.iloc[:,axis[0]:axis[1]])

In [36]:
path_data = r'src\data\ready\dataset-radius-1.5cm.csv'

extract = Extract()

extract._path_project = os.path.abspath(Path().resolve().parent)

dataset  = extract.dataframe(path_data)


y = partition(dataset,[0,4])
x = partition(dataset,[5,11])


### Análise da base de dados

In [37]:
print(f'Atributos e saídas:\n {dataset.columns}')
print(f"\nTamanho saída: {y.shape}\nTamanho entrada: {x.shape}\n")
print(f"Primeiras sáidas:\n{y[0:3]}\n")
print(f"Primeiras entrada:\n{x[0:3]}\n")


Atributos e saídas:
 Index(['theta_1', 'theta_2', 'theta_3', 'theta_4', 'theta_5', 'p_x', 'p_y',
       'p_z', 'roll', 'pich', 'yaw'],
      dtype='object')

Tamanho saída: (5818, 4)
Tamanho entrada: (5818, 6)

Primeiras sáidas:
[[ 0  0 12  0]
 [ 0 12 12  0]
 [ 0 12 24  0]]

Primeiras entrada:
[[ 5.32133136e+01 -2.47712141e-15  2.51517913e+00  3.14159265e+00
  -1.36135682e+00  2.75203954e-16]
 [ 5.36066568e+01 -1.78965176e-15  1.37424104e+01  2.37853137e+00
  -1.57079633e+00  7.63061284e-01]
 [ 5.28199704e+01 -3.12453009e-15  6.25758957e+00  3.14159265e+00
  -1.36135682e+00  2.37151833e-16]]



## Pré-processamento dos dados

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

def split(x, y, test_size):
    return train_test_split(x, y, test_size=test_size)

def zscore(dt_train,dt_test):
    scaler = StandardScaler()
    dt_train = scaler.fit_transform(dt_train)
    dt_test = scaler.transform(dt_test)
    
    return dt_train, dt_test, scaler

def statics_data_split(scaler):
    return DataFrame({'Média': scaler.mean_, 'Desvio Padrão': scaler.scale_})



In [39]:
size_train = 0.7
size_val = 0.2
size_test = 0.1

x_train, x_test, y_train, y_test = split(x,y,test_size=size_test)


x_train, x_test, scaler_x = zscore(x_train,x_test)

y_train, y_test, scaler_y = zscore(y_train,y_test)

#### info

In [40]:
statics_x = statics_data_split(scaler_x)
statics_x

,Média,Desvio Padrão
0,8.875359,20.762806
1,15.307506,22.534637
2,32.514692,16.372622
3,0.599810,1.687942
4,-0.572902,1.013675
5,-0.826993,1.721084


In [41]:
statics_y = statics_data_split(scaler_y)
statics_y

,Média,Desvio Padrão
0,60.814553,36.519198
1,80.763942,32.319595
2,59.812070,34.980490
3,33.971734,36.280578


#### Análises

In [42]:
print(f"treino: {size_train}\nTeste: {size_test}\nValidação: {size_val}")
print(f"Tamanho dos dados de treino: {len(x_train)}")
print(f"Tamanho dos dados de teste: {len(x_test)}")

treino: 0.7
Teste: 0.1
Validação: 0.2
Tamanho dos dados de treino: 5236
Tamanho dos dados de teste: 582


## Modelos

In [43]:
from kinematicsrobotics.model import Model

#### Informações

In [13]:
mlp = Model()
mlp.params
# model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': True,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 1000,
 'momentum': 0.9,
 'n_iter_no_change': 5,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 42,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

## Estimação dos hiperparâmetros

In [83]:
from sklearn.model_selection import ShuffleSplit,RandomizedSearchCV,GridSearchCV
from itertools import product
from sklearn.metrics import mean_squared_error,r2_score
from pandas import DataFrame

# Calcula a porcentagem relativa dos dados de treino e validação
def size_split(size_train,size_val):
    size_val = (1 - size_train/(size_train+size_val))
    return size_val

# Espaço de busca dos neurônios
def space_hidden(min_neurons,max_neurons,layers,step):
    # Lista que armazena os neurônios
    neuron_space = []
    
    # Gera a lista de possíveis números de neurônios em cada camada
    possible_neurons = list(range(min_neurons, max_neurons + 1,step))
    
    # Crie todas as combinações possíveis
    for num_layers in layers:
        neuron_combinations = list(product(possible_neurons, repeat=num_layers))
        neuron_space.extend(neuron_combinations)

    return neuron_space

# Parâmetro da rede mlp
def parameter_mlp(min_neurons,max_neurons,layers,step,func_act): 
    # Dicionário que armazena os hiperparâmetros
    param_grid = {
        'hidden_layer_sizes': space_hidden(min_neurons, max_neurons,layers,step),
        'activation': func_act
    }

    return param_grid

# Random Search
def RandomizedSearch(x, y, model, param_grid, scoring,cv, n_iter):
    # Configura os parâmetros da técnica de otimização 
    random_search = RandomizedSearchCV(estimator=model, 
                                       param_distributions=param_grid, 
                                       scoring=scoring, 
                                       cv=cv, 
                                       n_iter=n_iter, 
                                       random_state=42, 
                                       return_train_score=True,
                                       verbose=0)
    
    # Treina os modelos
    random_search.fit(x, y)

    # DataFrame que armazena os resultado dos hiperparâmetros
    history = DataFrame(random_search.cv_results_)

    # Melhor hiperparâmetro
    best_estimator = random_search.best_estimator_

    return history,best_estimator

# Grid Search
def GridSearch(x, y, model, param_grid, scoring, cv):
    # Configura os parâmetros da técnica de otimização 
    gridshearch = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               cv=cv,
                               scoring=scoring,
                               verbose=0,
                               return_train_score=True)
    
    # Treina os modelos
    gridshearch.fit(x, y)
    
    # Armazena os resultado dos hiperparâmetros
    history = DataFrame(gridshearch.cv_results_)
    
    best_estimator = gridshearch.best_estimator_

    return history,best_estimator

# Validação cruzada hold out
def holdout(n_splits,test_size):
    return ShuffleSplit(n_splits=n_splits, 
                      test_size=test_size, 
                      random_state=42)



In [10]:
# Salvar as métricas 
sv = save(path_project[0])
path_data_save =  r'src\data\ready\history.csv'

# Dados de treino e validação
size_val = size_split(size_train,size_val)

# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)


# validação cruzada hold out
n_splits = 5

# Número de amostras selecionadas no espaço de busca
n_iter = 1000

# Técnica de validação
cv = holdout(n_splits,size_val)

# inicializando a rede
mlp = MLP()

# APlicação do grid search
history,best_model = RandomizedSearch(x=x_train, 
                           y=y_train, 
                           model=mlp, 
                           param_grid=param_grid,
                           scoring = 'neg_mean_squared_error', 
                           cv=cv,
                           n_iter = n_iter)


sv.dataframe(history,path_data_save)



### Infomações

In [22]:
print(f"Número de parametros de camada oculta: {len(param_grid['hidden_layer_sizes'])}")
print(f"Atributos do retorno da gridsearsh: {history.columns}")

Número de parametros de camada oculta: 204183
Atributos do retorno da gridsearsh: Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_hidden_layer_sizes', 'param_activation', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'split0_train_score',
       'split1_train_score', 'split2_train_score', 'split3_train_score',
       'split4_train_score', 'mean_train_score', 'std_train_score'],
      dtype='object')


In [26]:
best_model

MLPRegressor(early_stopping=True, hidden_layer_sizes=(220, 120, 210, 210),
             max_iter=1000, n_iter_no_change=5, random_state=42)

In [27]:
history[['mean_test_score','std_test_score','mean_train_score', 'std_train_score','params']]

,mean_test_score,std_test_score,mean_train_score,std_train_score,params
0,-0.005603,0.000387,-0.004849,0.000581,"{'hidden_layer_sizes': (220, 120, 210, 210), '..."


## Métricas de Avaliação

In [85]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def predict_mse(y_predic,Y_test):    
    return mean_squared_error(Y_test, y_predic,multioutput='raw_values')



def curve_loss(model):
    plt.plot(model.loss_curve_)
    plt.title('Erro Quadratico')
    plt.xlabel('Épocas')
    plt.ylabel('MSE')
    plt.show()

def robo():
    # Define os parâmetros DH para cada elo
    Elos = [['theta_1',10,0,90,0],
            ['theta_2',0,18,180,0],
            ['theta_3',0,18,-180,0],

            ['theta_4',0,0,90,90],
            ['theta_5',18,0,0,0]]

    # Cria o robo por meio dos parâmetros
    return Robo('Robotest',Elos)



## Treinamento, validação e teste do modelo selecionado

In [44]:
path_data = r'src\data\ready\history.csv'
history = extract.dataframe(path_data)

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

In [47]:
# Treina o modelo com hiperparâmetros selecionados
mlp = Model()

mlp.set_model(**params)

mlp.fit(x = x_train,y = y_train)



In [48]:
mlp.model

MLPRegressor(early_stopping=True, hidden_layer_sizes=(120, 220, 290),
             max_iter=1000, n_iter_no_change=5, random_state=42)

### Espaço das juntas

In [88]:
# Metrica de dados de treino
y_predic = scaler_y.inverse_transform(mlp.predict(x_train))
y_train = scaler_y.inverse_transform(y_train)
predict_mse(y_predic, y_train)

array([ 1.03045163,  4.40377084, 10.42354988,  6.19330727])

In [89]:
# Metrica dos dados de teste
y_predic = scaler_y.inverse_transform(mlp.predict(x_test))
y_test = scaler_y.inverse_transform(y_test)
predict_mse(y_predic, y_test)

array([ 1.09959474,  5.41123919, 13.74559807,  7.01453628])

### Espaço Operacional

In [91]:
def transfor(array):
    novo_array = []
    for sub_array in array:
            novo_sub_array = append(sub_array, 0)
            novo_array.append(novo_sub_array)
    return novo_array
x_test = scaler_x.inverse_transform(x_test)

In [92]:
pose_predic = transfor(y_predic)

pose_real = DataFrame(x_test, columns=['p_x','p_y', 'p_z','roll','pich','yaw'])
pose_real = pose_real[['p_x','p_y', 'p_z']]
pose_real

,p_x,p_y,p_z
0,-3.752096,4.288665e+01,22.009608
1,12.039621,2.581905e+01,51.858136
2,-24.671478,-1.424408e+01,47.232262
3,-34.443818,1.776357e-15,48.498174
4,20.677811,2.464286e+01,38.965064
...,...,...,...
577,23.844095,2.841629e+01,20.416573
578,7.543940,4.278381e+01,1.732802
579,12.039621,-2.581905e+01,39.747441
580,25.462605,3.034515e+01,23.540168


In [93]:
data = mapping.operational_space(pose_predic)

In [94]:
dt_predic = DataFrame(data,columns=['R_11','R_12','R_13','p_x','R_21','R_22','R_23','p_y','R_31','R_32','R_33','p_z'])
dt = dt_predic[['p_x', 'p_y', 'p_z']]
dt

,p_x,p_y,p_z
0,-25.564833,-2.098390,-9.314471
1,-22.410147,10.963173,16.254111
2,6.100140,-9.916065,15.712246
3,0.461947,-15.878274,18.522460
4,-27.457807,23.545584,3.194996
...,...,...,...
577,-26.117746,21.603906,-15.326076
578,-10.614455,2.103847,-12.843672
579,13.494952,6.210928,7.062461
580,-21.652794,18.247551,-11.504055
